In [27]:
import sys
import importlib
import pyLDAvis.gensim
import pyLDAvis
import pandas as pd


sys.path.append('./codes')
from codes import LDA 
from codes import classification
import importlib
importlib.reload(LDA)
importlib.reload(classification)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from codes.LDA import *

from codes.classification import *


In [2]:
df = pd.read_csv('../amazon_data/Amazon_Fashion.csv')

In [46]:
df.size
df['rating'].value_counts()

rating
5.0    1411756
4.0     348881
1.0     300032
3.0     243118
2.0     171666
Name: count, dtype: int64

## Making well-balanced data

In [17]:
data = df[~df["text"].isna()]
data = data[data['text'].apply(lambda x: len(str(x).split()) > 10)]
sampled_per_rating = [data[data['rating'] == rating].sample(n=10000, random_state=42) for rating in range(1, 6)]
required_samples = pd.concat(sampled_per_rating, axis=0)
remaining_samples = data.drop(required_samples.index).sample(n=100000 - len(required_samples), random_state=42)
data = pd.concat([required_samples, remaining_samples], axis=0)
data = data[data['text'].apply(lambda x: len(str(x).split()) > 10)]
print(data['rating'].value_counts())

rating
5.0    36030
4.0    18004
1.0    16521
3.0    15569
2.0    13876
Name: count, dtype: int64


In [19]:
texts_bow, dictionary, id2token = preprocess_text(data, True, True)

---Preprocessing starting ---
---Tokenization done---
---Numbers removed---
---Two letter words removed---
---Written-out numbers removed---
---Verbs removed BIG---
---Lematization done---
---Stopwords removed---
---Choosed words removed---
---Bigrams done---
---Preprocessing done---


In [22]:
model, dictionary = LDA_training(data, False,9, 1000, 10, 200, 1, 10, texts_bow, dictionary, id2token)

--- LDA starting ---
--- LDA finished ---
Medium koherence of topics: -2.3864.
Topic 0: pair, year, old, glass, money, worth, son, year_old, summer, sunglass
Topic 1: product, receive, gift, light, item, foot, arrive, bracelet, without, weight
Topic 2: time, come, day, use, band, first, lot, work, break, easy
Topic 3: fit, love, size, order, buy, small, little, would, perfect, large
Topic 4: belt, comfy, boot, second, next, card, weather, first_time, many_compliment, buckle
Topic 5: back, say, send, amazon, ship, thank, review, read, seller, costume
Topic 6: color, well, make, comfortable, also, much, soft, enough, around, different
Topic 7: give, earring, star, ear, dry, real, silver, reason, ankle, shrink
Topic 8: great, look, good, quality, nice, really, material, cute, price, purchase


In [23]:
data_with_sentiment = add_sentiment_column(data)
feature_matrix = topic_distributions_to_matrix(model, texts_bow, 9)
labels = data_with_sentiment['sentiment_encoded']
X_train, X_test, y_train, y_test = train_test_spliter(feature_matrix, labels)

In [24]:
modelR, y_pred = train_and_evaluate_Random_Forest(feature_matrix, labels, X_train, X_test, y_train, y_test)

---Random Forest Training ---
Accuracy for Random Forest: 0.5372
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.43      0.30      0.35      6084
           1       0.20      0.04      0.06      3109
           2       0.58      0.82      0.68     10807

    accuracy                           0.54     20000
   macro avg       0.40      0.38      0.36     20000
weighted avg       0.47      0.54      0.48     20000





In [28]:
modelL, y_predL = train_and_evaluate_Logistic_Regression(feature_matrix, labels, X_train, X_test, y_train, y_test)

---Logistic Regression Training ---
Accuracy for Logistic Regression: 0.5466
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.54      0.06      0.10      6084
           1       0.00      0.00      0.00      3109
           2       0.55      0.98      0.70     10807

    accuracy                           0.55     20000
   macro avg       0.36      0.35      0.27     20000
weighted avg       0.46      0.55      0.41     20000





c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

# checking removing the 3 star reviews

In [29]:
data2 = df[~df["text"].isna()]
data2 = data2[data2['text'].apply(lambda x: len(str(x).split()) > 10)]
sampled_per_rating = [data2[data2['rating'] == rating].sample(n=10000, random_state=42) for rating in range(1, 6)]
required_samples = pd.concat(sampled_per_rating, axis=0)
remaining_samples = data2.drop(required_samples.index).sample(n=100000 - len(required_samples), random_state=42)
data2 = pd.concat([required_samples, remaining_samples], axis=0)
data2 = data2[data2['text'].apply(lambda x: len(str(x).split()) > 10)]
data2 = data2[data2["rating"]!=3]
print(data2['rating'].value_counts())

rating
5.0    36030
4.0    18004
1.0    16521
2.0    13876
Name: count, dtype: int64


In [30]:
texts_bow2, dictionary2, id2token2 = preprocess_text(data2, True, True)

---Preprocessing starting ---
---Tokenization done---
---Numbers removed---
---Two letter words removed---
---Written-out numbers removed---
---Verbs removed BIG---
---Lematization done---
---Stopwords removed---
---Choosed words removed---
---Bigrams done---
---Preprocessing done---


In [31]:
model2, dictionary2 = LDA_training(data2, False,9, 1000, 10, 200, 1, 10, texts_bow2, dictionary2, id2token2)

--- LDA starting ---
--- LDA finished ---
Medium koherence of topics: -2.4581.
Topic 0: fit, size, order, small, perfect, large, big, way, return, even
Topic 1: different, glass, black, white, blue, face, case, sunglass, lens, red
Topic 2: year, daughter, perfectly, old, fit_perfectly, foot, year_old, summer, costume, boot
Topic 3: wash, dry, cool, christmas, type, glad, house, help, shrink, base
Topic 4: quality, price, product, back, item, arrive, mask, amazon, package, send
Topic 5: great, look, would, well, color, good, nice, make, really, little
Topic 6: love, buy, time, purchase, come, beautiful, receive, gift, lot, easy
Topic 7: earring, ring, necklace, belt, ear, lose, chain, metal, jewelry, fast
Topic 8: pair, day, use, band, first, work, thing, pocket, try, need


In [32]:
data_with_sentiment2 = add_sentiment_column(data2)
feature_matrix2 = topic_distributions_to_matrix(model2, texts_bow2, 9)
labels2 = data_with_sentiment2['sentiment_encoded']
X_train2, X_test2, y_train2, y_test2 = train_test_spliter(feature_matrix2, labels2)

In [33]:
modelR2, y_pred2 = train_and_evaluate_Random_Forest(feature_matrix2, labels2, X_train2, X_test2, y_train2, y_test2)

---Random Forest Training ---
Accuracy for Random Forest: 0.6481
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.51      0.33      0.40      6033
           2       0.69      0.82      0.75     10854

    accuracy                           0.65     16887
   macro avg       0.60      0.58      0.58     16887
weighted avg       0.63      0.65      0.63     16887





In [34]:
modelL2, y_predL2 = train_and_evaluate_Logistic_Regression(feature_matrix2, labels2, X_train2, X_test2, y_train2, y_test2)

---Logistic Regression Training ---
Accuracy for Logistic Regression: 0.6532
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.56      0.13      0.21      6033
           2       0.66      0.94      0.78     10854

    accuracy                           0.65     16887
   macro avg       0.61      0.54      0.49     16887
weighted avg       0.63      0.65      0.58     16887





# Trying on huge well-balanced data

In [55]:
dataH = df[~df["text"].isna()]
dataH = dataH[dataH['text'].apply(lambda x: len(str(x).split()) > 10)]
sampled_per_rating = [dataH[dataH['rating'] == rating].sample(n=100000, random_state=42) for rating in range(1, 6)]
required_samples = pd.concat(sampled_per_rating, axis=0)
remaining_samples = dataH.drop(required_samples.index).sample(n=1000000 - len(required_samples), random_state=42)
dataH = pd.concat([required_samples, remaining_samples], axis=0)
dataH = dataH[dataH['text'].apply(lambda x: len(str(x).split()) > 10)]
print(data['rating'].value_counts())

rating
5.0    36030
4.0    18004
1.0    16521
3.0    15569
2.0    13876
Name: count, dtype: int64


In [56]:
texts_bowH, dictionaryH, id2tokenH = preprocess_text(dataH, True, True)

---Preprocessing starting ---
---Tokenization done---
---Numbers removed---
---Two letter words removed---
---Written-out numbers removed---
---Verbs removed BIG---
---Lematization done---
---Stopwords removed---
---Choosed words removed---
---Bigrams done---
---Preprocessing done---


In [58]:
modelH, dictionaryH = LDA_training(dataH, False, 9, 1000, 2, 100, 1, 10, texts_bowH, dictionaryH, id2tokenH)

--- LDA starting ---
--- LDA finished ---
Medium koherence of topics: -2.3701.
Topic 0: easy, work, pocket, thing, glass, without, set, new, best, case
Topic 1: fit, size, buy, love, order, well, would, perfect, small, little
Topic 2: year, old, year_old, bra, glove, awesome, purse, area, fast, especially
Topic 3: time, come, day, band, compliment, first, many, break, strap, use
Topic 4: stay, mask, face, hard, easily, leather, place, dry, night, plastic
Topic 5: pair, black, foot, white, blue, boot, christmas, excellent, one, match
Topic 6: back, earring, ring, necklace, piece, bracelet, belt, ear, nicely, chain
Topic 7: great, love, look, good, color, nice, quality, really, make, comfortable
Topic 8: product, review, thank, amazon, ship, seller, receive, wait, company, stylish


In [59]:
data_with_sentimentH = add_sentiment_column(dataH)
feature_matrixH = topic_distributions_to_matrix(modelH, texts_bowH, 9)
labelsH = data_with_sentimentH['sentiment_encoded']
X_trainH, X_testH, y_trainH, y_testH = train_test_spliter(feature_matrixH, labelsH)

In [60]:
modelRH, y_predRH = train_and_evaluate_Random_Forest(feature_matrixH, labelsH, X_trainH, X_testH, y_trainH, y_testH)

---Random Forest Training ---
Accuracy for Random Forest: 0.6046
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.47      0.27      0.35     52857
           1       0.20      0.03      0.05     27737
           2       0.64      0.88      0.74    119406

    accuracy                           0.60    200000
   macro avg       0.44      0.40      0.38    200000
weighted avg       0.53      0.60      0.54    200000





In [68]:
modelLH, y_predLH = train_and_evaluate_Logistic_Regression(feature_matrixH, labelsH, X_trainH, X_testH, y_trainH, y_testH)

---Logistic Regression Training ---
Accuracy for Logistic Regression: 0.6074
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.56      0.09      0.16     52857
           1       0.00      0.00      0.00     27737
           2       0.61      0.98      0.75    119406

    accuracy                           0.61    200000
   macro avg       0.39      0.36      0.30    200000
weighted avg       0.51      0.61      0.49    200000





c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [70]:
modelGH, y_predGH = train_and_evaluate_Gradient_Boosting(feature_matrixH, labelsH, X_trainH, X_testH, y_trainH, y_testH)

--- Gradient Boosting Training ---
Accuracy for Gradient Boosting: 0.6130
Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           0       0.55      0.14      0.23     52857
           1       0.00      0.00      0.00     27737
           2       0.62      0.96      0.75    119406

    accuracy                           0.61    200000
   macro avg       0.39      0.37      0.33    200000
weighted avg       0.51      0.61      0.51    200000





c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

# -||- without 3 stars

In [63]:
data2H = df[~df["text"].isna()]
data2H = data2H[data2H['text'].apply(lambda x: len(str(x).split()) > 10)]
sampled_per_rating = [data2H[data2H['rating'] == rating].sample(n=100000, random_state=42) for rating in range(1, 6)]
required_samples = pd.concat(sampled_per_rating, axis=0)
remaining_samples = data2H.drop(required_samples.index).sample(n=1000000 - len(required_samples), random_state=42)
data2H = pd.concat([required_samples, remaining_samples], axis=0)
data2H = data2H[data2H['text'].apply(lambda x: len(str(x).split()) > 10)]
data2H = data2H[data2H["rating"]!=3]
print(data2H['rating'].value_counts())

rating
5.0    427346
4.0    169729
1.0    150564
2.0    113742
Name: count, dtype: int64


In [64]:
texts_bow2H, dictionary2H, id2token2H = preprocess_text(data2H, True, True)

---Preprocessing starting ---
---Tokenization done---
---Numbers removed---
---Two letter words removed---
---Written-out numbers removed---
---Verbs removed BIG---
---Lematization done---
---Stopwords removed---
---Choosed words removed---
---Bigrams done---
---Preprocessing done---


In [65]:
model2H, dictionary2H = LDA_training(data2H, False, 9, 1000, 2, 100, 1, 10, texts_bow2H, dictionary2H, id2token2H)

--- LDA starting ---
--- LDA finished ---
Medium koherence of topics: -2.6863.
Topic 0: time, day, come, use, band, easy, work, first, pocket, strap
Topic 1: love, fit, size, buy, order, perfect, would, little, small, cute
Topic 2: wash, ring, money, highly, highly_recommend, dry, jewelry, gold, water, green
Topic 3: pair, year, compliment, light, old, glass, earring, son, weight, year_old
Topic 4: back, also, right, review, enough, could, around, give, side, without
Topic 5: great, look, well, good, nice, quality, make, really, material, price
Topic 6: purchase, product, gift, happy, receive, item, arrive, set, thank, new
Topic 7: color, different, black, mask, face, blue, white, sunglass, show, tie
Topic 8: many, warm, nicely, favorite, heavy, durable, ever, excellent, neck, many_compliment


In [66]:
data_with_sentiment2H = add_sentiment_column(data2H)
feature_matrix2H = topic_distributions_to_matrix(model2H, texts_bow2H, 9)
labels2H = data_with_sentiment2H['sentiment_encoded']
X_train2H, X_test2H, y_train2H, y_test2H = train_test_spliter(feature_matrix2H, labels2H)

In [67]:
modelRH2, y_predRH2 = train_and_evaluate_Random_Forest(feature_matrix2H, labels2H, X_train2H, X_test2H, y_train2H, y_test2H)

---Random Forest Training ---
Accuracy for Random Forest: 0.7146
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.56      0.31      0.40     52869
           2       0.75      0.89      0.81    119408

    accuracy                           0.71    172277
   macro avg       0.65      0.60      0.61    172277
weighted avg       0.69      0.71      0.69    172277





In [69]:
modelL2H, y_predL2H = train_and_evaluate_Logistic_Regression(feature_matrix2H, labels2H, X_train2H, X_test2H, y_train2H, y_test2H)

---Logistic Regression Training ---
Accuracy for Logistic Regression: 0.6986
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.56      0.08      0.14     52869
           2       0.71      0.97      0.82    119408

    accuracy                           0.70    172277
   macro avg       0.63      0.53      0.48    172277
weighted avg       0.66      0.70      0.61    172277





In [71]:
modelGH2, y_predGH2 = train_and_evaluate_Gradient_Boosting(feature_matrix2H, labels2H, X_train2H, X_test2H, y_train2H, y_test2H)

--- Gradient Boosting Training ---
Accuracy for Gradient Boosting: 0.7110
Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           0       0.62      0.15      0.24     52869
           2       0.72      0.96      0.82    119408

    accuracy                           0.71    172277
   macro avg       0.67      0.55      0.53    172277
weighted avg       0.69      0.71      0.64    172277



